In [0]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_file= pd.read_csv(r'/content/drive/My Drive/DL/train.csv')
test_file=pd.read_csv(r'/content/drive/My Drive/DL/test.csv')

In [60]:
test_file.shape

(1459, 80)

In [0]:
test_file['SalePrice']=np.nan

In [8]:
test_file.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,...,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,NaN
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,...,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,NaN
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,...,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,NaN
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,...,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,NaN
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,...,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal,NaN


In [0]:
train_file['data']='train'
test_file['data']='test'

In [0]:
test_file=test_file[train_file.columns]

In [0]:
file=pd.concat([train_file,test_file],axis=0)

In [24]:
file.dtypes

Id                 int64
MSSubClass         int64
MSZoning         float64
LotFrontage      float64
LotArea            int64
                  ...   
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice        float64
data              object
Length: 82, dtype: object

In [0]:
file['MSZoning']=pd.to_numeric(file['MSZoning'],errors='coerce')


In [40]:
file.isnull().sum()

Id                  0
MSSubClass          0
MSZoning         2919
LotFrontage         0
LotArea             0
                 ... 
YrSold              0
SaleType            1
SaleCondition       0
SalePrice           0
data                0
Length: 82, dtype: int64

In [57]:
file.isnull().sum()

MSSubClass          0
MSZoning         2919
LotFrontage         0
LotArea             0
OverallQual         0
                 ... 
GarageCond_Po       0
GarageCond_TA       0
Fence_GdWo          0
Fence_MnPrv         0
Fence_MnWw          0
Length: 146, dtype: int64

In [66]:

def fill_empty(file):
    file.fillna(value = file.mean(), inplace = True)
    return file.isnull().sum()

fill_empty(file)
fill_empty(train_file)

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
data             0
Length: 82, dtype: int64

In [0]:
file[]

In [0]:
file.drop(['Id','MSZoning','FireplaceQu','SaleCondition','SaleType','MiscFeature','PoolQC','PavedDrive','GarageFinish','Functional','Alley','LotShape','LandContour','LotConfig','LandSlope','Condition1','Condition2','Exterior1st','Exterior2nd','MasVnrType','MasVnrArea','Foundation'],axis=1,inplace=True)

In [69]:
cat_vars=file.select_dtypes(['object']).columns

cat_vars

Index(['Street', 'Utilities', 'Neighborhood', 'BldgType', 'HouseStyle',
       'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'GarageType',
       'GarageQual', 'GarageCond', 'Fence', 'data'],
      dtype='object')

In [70]:
for col in cat_vars[:-1]:
    dummy=pd.get_dummies(file[col],drop_first=True,prefix=col)
    file=pd.concat([file,dummy],axis=1)
    del file[col]
    print(col)
del dummy

Street
Utilities
Neighborhood
BldgType
HouseStyle
RoofStyle
RoofMatl
ExterQual
ExterCond
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
GarageType
GarageQual
GarageCond
Fence


In [44]:
file.dtypes

MSSubClass         int64
MSZoning         float64
LotFrontage      float64
LotArea            int64
OverallQual        int64
                  ...   
GarageCond_Po      uint8
GarageCond_TA      uint8
Fence_GdWo         uint8
Fence_MnPrv        uint8
Fence_MnWw         uint8
Length: 146, dtype: object

In [45]:
file.shape

(2919, 146)

In [72]:
train_file=file[file['data']=='train']
test_file=file[file['data']=='test']

train_file.drop(columns='data',axis=1,inplace=True)
test_file.drop(columns=['data','SalePrice'],axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [73]:
train_file.isnull().sum()

MSSubClass       0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
                ..
GarageCond_Po    0
GarageCond_TA    0
Fence_GdWo       0
Fence_MnPrv      0
Fence_MnWw       0
Length: 144, dtype: int64

In [0]:
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import RandomizedSearchCV

In [75]:
training_data, validation_data = train_test_split(train_file,test_size=0.2,random_state=10)
print(training_data.shape)
print(validation_data.shape)

(1168, 144)
(292, 144)


In [0]:
x_train=training_data.drop(['SalePrice'],axis=1)
y_train=training_data['SalePrice']

In [0]:
lm=LinearRegression()

0

In [78]:
lm.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [80]:
testing_data=validation_data.drop('SalePrice',axis=1)
predicted_ir=lm.predict(testing_data)
MAE=mean_absolute_error(validation_data['SalePrice'],predicted_ir)
MAE

22221.106538154436

In [81]:
list(zip(x_train.columns,lm.coef_))

[('MSSubClass', -87.72165559221999),
 ('LotFrontage', 63.7046102124094),
 ('LotArea', 0.4742519556865654),
 ('OverallQual', 6867.9776015402485),
 ('OverallCond', 6215.233684584495),
 ('YearBuilt', 325.85909785886764),
 ('YearRemodAdd', 66.19706692212428),
 ('BsmtFinSF1', 13.355497645055836),
 ('BsmtFinSF2', 7.379749183663044),
 ('BsmtUnfSF', 3.399291405426453),
 ('TotalBsmtSF', 24.13452376372169),
 ('1stFlrSF', 7.40489368981946),
 ('2ndFlrSF', 41.00901074859496),
 ('LowQualFinSF', -22.405309094237055),
 ('GrLivArea', 26.00843684187066),
 ('BsmtFullBath', 3477.492320504939),
 ('BsmtHalfBath', 221.1227471952393),
 ('FullBath', 3921.6788868423073),
 ('HalfBath', -544.9124671871732),
 ('BedroomAbvGr', -4093.203291937247),
 ('KitchenAbvGr', -25693.11381322831),
 ('TotRmsAbvGrd', 3079.213704097294),
 ('Fireplaces', 2964.4604448235295),
 ('GarageYrBlt', 14.425301382067119),
 ('GarageCars', 7473.382224413803),
 ('GarageArea', 6.575505765894377),
 ('WoodDeckSF', 17.511215553983675),
 ('OpenPorc

In [0]:
lambdas=np.linspace(0,1,20)

In [0]:
params={'alpha':lambdas}

In [0]:
ridge_model=Ridge(fit_intercept=True)

In [0]:
grid_search_rd=GridSearchCV(ridge_model,param_grid=params,cv=10,scoring='neg_mean_absolute_error')

In [122]:
grid_search_rd.fit(x_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=0)

In [123]:
grid_search_rd.best_estimator_
#alpha=88

Ridge(alpha=0.05263157894736842, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [0]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [109]:
report(grid_search.cv_results_,3)

Model with rank: 1
Mean validation score: -18564.598 (std: 1027.561)
Parameters: {'alpha': 8.105263157894736}

Model with rank: 2
Mean validation score: -18564.736 (std: 1035.204)
Parameters: {'alpha': 8.578947368421051}

Model with rank: 3
Mean validation score: -18564.969 (std: 1042.715)
Parameters: {'alpha': 9.052631578947368}



In [127]:
testing_data=validation_data.drop('SalePrice',axis=1)
predicted=grid_search_rd.predict(testing_data)
MAE=mean_absolute_error(validation_data['SalePrice'],predicted)
MAE

20731.048574245317

20731.048574245317

In [0]:
lambdas=np.linspace(12,20,50)

In [0]:
model1=Lasso(fit_intercept=True)
params={'alpha':lambdas}

In [0]:
grid_search_cv=GridSearchCV(model1,param_grid=params,cv=10,scoring='neg_mean_absolute_error')

In [149]:
grid_search_cv.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1025876888.5875244, tolerance: 671934674.4957632
  positive)


GridSearchCV(cv=10, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([12.        , 12.16326531, 12.32653061, 12.48979592, 12.65306122,
       12.81632653, 12.9795918...
       16.08163265, 16.24489796, 16.40816327, 16.57142857, 16.73469388,
       16.89795918, 17.06122449, 17.2244898 , 17.3877551 , 17.55102041,
       17.71428571, 17.87755102, 18.04081633, 18.20408163, 18.36734694,
       18.53061224, 18.69387755, 18.85714286, 19.02040816, 19.18367347,
       19.34693878, 19.51020408, 19.67346939, 19.83673469, 20.        ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_erro

In [150]:
lasso_model=grid_search_cv.best_estimator_
lasso_model

Lasso(alpha=17.714285714285715, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [0]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [152]:
report(grid_search_cv.cv_results_,3)

Model with rank: 1
Mean validation score: -18022.265 (std: 1432.867)
Parameters: {'alpha': 17.714285714285715}

Model with rank: 2
Mean validation score: -18022.296 (std: 1429.955)
Parameters: {'alpha': 17.877551020408163}

Model with rank: 3
Mean validation score: -18022.303 (std: 1435.839)
Parameters: {'alpha': 17.551020408163264}



In [153]:
lasso_model.fit(x_train,y_train)

Lasso(alpha=17.714285714285715, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [0]:
pred = grid_search.predict(testing_data)

In [156]:
mean_absolute_error(validation_data['SalePrice'],pred)
#808.9139069064713

19059.22102565526

In [157]:
#Generating Random Parameters

# Number of trees in random forest
n_estimators = 200,2000,10

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [10, 110, 11]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid

{'bootstrap': [True, False],
 'max_depth': [10, 110, 11, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': (200, 2000, 10)}

In [158]:
# Use the random grid to search for best hyperparameters



# First create the base model to tune
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   52.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.4min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [159]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 200}

In [0]:
#Running RandomForestRegressor with best parameters

rf = RandomForestRegressor(n_estimators=200, bootstrap=False, max_depth=None, max_features='sqrt', min_samples_leaf=2, min_samples_split=2)

In [161]:
rf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
pred = rf.predict(testing_data)

In [164]:
mean_absolute_error(validation_data['SalePrice'],pred)
#808.9139069064713

17661.36321020464

In [0]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

In [166]:
feature_importances

,importance
OverallQual,0.101831
GarageCars,0.065914
GrLivArea,0.063822
YearBuilt,0.060428
1stFlrSF,0.055756
...,...
Neighborhood_Blueste,0.000000
BsmtCond_Po,0.000000
RoofMatl_Metal,0.000000
RoofMatl_Roll,0.000000


In [0]:
pred = rf.predict(testing_data)

In [169]:
mean_absolute_error(validation_data['SalePrice'],pred)
#741.3033985923656

17661.36321020464

In [170]:
#Running the model on entire train_file

xtrain1 = train_file.drop('SalePrice', axis=1)
ytrain2 = train_file['SalePrice']

rf.fit(xtrain1,ytrain2)

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
predicted_values = rf.predict(test_file)

In [172]:
predicted_values[0:5]

array([126877.80534843, 156860.76040821, 179918.11911102, 189897.52622401,
       193373.53002931])

In [0]:
#Submission File
pd.DataFrame(predicted_values).to_csv("mysubmission.csv",index=False)

In [168]:
predicted_values[0:5]

array([189000.14308845, 199236.08863178, 128877.53212816, 158497.13399123,
       309869.29712269, 284469.27388676, 214515.93442189, 122440.98289313,
       195501.60943357, 158315.37614989, 151469.893686  , 161277.26768732,
       261665.02599711, 130069.00306818, 190301.26966768, 177616.72320617,
       268149.39299998, 143611.49324541, 197387.26925513, 188625.10095238,
       379298.02441743, 127941.41355159, 133805.23754949, 118693.20339286,
       186068.19062807, 161399.75994751, 173249.75095238,  96147.20718975,
       147985.96031746, 188402.19887623, 116738.62578555, 154497.06741913,
       128506.80144311, 131871.32405892, 146330.52094517,  80945.45833333,
       143620.03519462, 217900.6285372 , 314811.53369803, 145567.17861416,
       212769.67921678, 122534.78151515, 110453.68815629, 153786.43113095,
       206868.65303905, 195680.72810693, 244306.93940693, 143950.81615711,
       126177.63900963, 165661.06252771, 194292.37831298,  90493.80809975,
       185478.67838525, 1

In [0]:
mean_absolute_error(validation_data['Counterfeit_Sales'],pred)
#741.3033985923656